In [29]:
from functions import *
import pandas as pd
import seaborn as sns
import numpy as np
from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter
import re
import ppscore as pps

In [41]:
df_training = pd.read_csv("training_data.csv")
df_test = pd.read_csv("forecast_starting_data.csv")
df_macro = pd.read_csv("macro_data.csv")

C:\Users\devas\AppData\Local\Temp\ipykernel_28832\998706994.py:1: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  df_training = pd.read_csv("training_data.csv")


# Initial data cleaning and sample creation
To do some preliminary cleaning of the data, we dropped columns with only NaN values. Then, we created our sample dataset that we used to identify relationships and train the model. 50% of this sample contained all the accounts that were charged off in the dataset (15698 values), while the other 50% of this sample contained a random sample of 15698 accounts that weren't charged off in the dataset, for a total of 31396 accounts.

In [42]:
df_training.drop('charge_off_reason_code', axis=1, inplace=True)
df_training.drop('writeoff_date', axis =1, inplace=True)
dataless_forecast_cols = []
for column in df_test.columns:
    if ((len(df_test[column].unique()) == 1)):
        df_test.append(column)
df_training.drop(dataless_forecast_cols, axis=1, inplace=True)

C:\Users\devas\AppData\Local\Temp\ipykernel_28832\3165823135.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test.append(column)


In [ ]:
df_training

In [34]:
df_no_charge_sample = df_training[df_training['charge_off'] == 0].sample(n=15698)
df_charge = df_training[df_training['charge_off'] == 1]
df_tot_sample = pd.concat([df_charge, df_no_charge_sample], axis=0)

In [35]:
data = []
for column in df_tot_sample.columns:
    data.append(pps.score(df_tot_sample, x=column, y="charge_off"))
df_random_pps1 = pd.DataFrame(data)
df_random_pps1 = df_random_pps1.sort_values(by='ppscore', ascending=False).head(n=20)
df_random_pps1

,x,y,ppscore,case,is_valid_score,metric,baseline_score,model_score,model
0,financial_active,charge_off,0.165799,regression,True,mean absolute error,0.488,0.407090,DecisionTreeRegressor()
1,net_payment_behaviour_tripd,charge_off,0.399867,regression,True,mean absolute error,0.488,0.292865,DecisionTreeRegressor()
2,promotion_flag,charge_off,0.014134,regression,True,mean absolute error,0.488,0.481103,DecisionTreeRegressor()
3,variable_rate_index,charge_off,0.000000,regression,True,mean absolute error,0.488,0.499084,DecisionTreeRegressor()
4,account_status_code,charge_off,0.148939,regression,True,mean absolute error,0.488,0.415318,DecisionTreeRegressor()
...,...,...,...,...,...,...,...,...,...
62,due_balance_7,charge_off,0.000000,regression,True,mean absolute error,0.488,0.498668,DecisionTreeRegressor()
63,due_balance_8,charge_off,0.000000,regression,True,mean absolute error,0.488,0.499763,DecisionTreeRegressor()
64,snapshot,charge_off,0.000000,regression,True,mean absolute error,0.488,0.497284,DecisionTreeRegressor()
65,mth_code,charge_off,0.000000,regression,True,mean absolute error,0.488,0.499445,DecisionTreeRegressor()


# Feature engineering
We incorporate the macro data into our DataFrame to take economic data into cosideration when making our prediction and constructing our model. Then, we did feature engineering to find the survival times, assign each month a number ranging from 1 to 24, sort the data values, and remove multiple occurences of same observations so that only the observations in which people are charged off or in which the last observations of the people which don't get charged off are remaining. This prepares the data to be applied to the Cox Proportional Hazards Model.

In [ ]:
macro_final = pd.DataFrame()
for i, row in df_macro.iterrows():
    if re.search(r"2017$", str(row["Mnemonic"])) or re.search(r"2018$", str(row["Mnemonic"])) or re.search(r"2019$", str(row["Mnemonic"])) or re.search(r"2020$", str(row["Mnemonic"])) or re.search(r"2021$", str(row["Mnemonic"])):
        t1 = pd.DataFrame(row).T
        month_code = format_date(t1['Mnemonic'].iloc[0])
        t1['mth'] = month_code
        macro_final = pd.concat([macro_final, t1], axis=0)

In [38]:
x = pd.DataFrame()
final = pd.DataFrame()
a = 0 #
for i in df_tot_sample['mth_code'].unique():
    left_block = df_tot_sample[df_tot_sample['mth_code'] == i]
    temp2 = macro_final[macro_final['mth'] == ((str)(i))]
    right_block = pd.concat([temp2]*len(left_block))
    
    left_block = left_block.reset_index(drop=True)
    right_block = right_block.reset_index(drop=True)
    
    x = pd.concat([left_block, right_block], axis=1).reset_index()
    if a == 0:
        final = x
        a+=1
    else: 
        final = pd.concat([x, final], axis=0)
final

,index,financial_active,net_payment_behaviour_tripd,promotion_flag,variable_rate_index,account_status_code,active_12_mths,bank_fico_buckets_20,mob,open_closed_flag,...,M_FETP.IUSA,M_FLTV.IUSA,M_FC$.IUSA,M_FEIAMGTTQ.IUSA,M_FXCPMGASUQ.IUSA,M_FRBBBQ.IUSA,M_FHOWNRQ.IUSA,M_FIFR_FEDS.IUSA,M_FIFR$_FEDS.IUSA,mth
0,0,0.0,P,0,0,A,1.0,761+,19,1.0,...,125.317,79.5,12775.504,2.6712,253.46,4.058025114,64.18299722,782.9992665,616.3053138,201801
1,1,1.0,I,0,1,A,1.0,761+,122,1.0,...,125.317,79.5,12775.504,2.6712,253.46,4.058025114,64.18299722,782.9992665,616.3053138,201801
2,2,0.0,T,1,1,A,1.0,661-680,14,1.0,...,125.317,79.5,12775.504,2.6712,253.46,4.058025114,64.18299722,782.9992665,616.3053138,201801
3,3,1.0,I,0,0,A,1.0,761+,26,1.0,...,125.317,79.5,12775.504,2.6712,253.46,4.058025114,64.18299722,782.9992665,616.3053138,201801
4,4,1.0,T,1,1,I 7-12,1.0,641-660,21,1.0,...,125.317,79.5,12775.504,2.6712,253.46,4.058025114,64.18299722,782.9992665,616.3053138,201801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,194,1.0,P,0,0,A,0.0,761+,12,1.0,...,125.672,79.3,12765.158,2.70475,256.9375,4.206673162,64.29943946,783.2181714,611.3780129,201802
195,195,1.0,I,1,0,A,1.0,701-720,27,1.0,...,125.672,79.3,12765.158,2.70475,256.9375,4.206673162,64.29943946,783.2181714,611.3780129,201802
196,196,1.0,P,1,0,A,1.0,761+,41,1.0,...,125.672,79.3,12765.158,2.70475,256.9375,4.206673162,64.29943946,783.2181714,611.3780129,201802
197,197,1.0,P,0,0,A,1.0,701-720,65,1.0,...,125.672,79.3,12765.158,2.70475,256.9375,4.206673162,64.29943946,783.2181714,611.3780129,201802


In [39]:
data = []
for column in final.columns:
    data.append(pps.score(final, x=column, y="charge_off"))
df_random_pps = pd.DataFrame(data)
df_random_pps = df_random_pps.sort_values(by='ppscore', ascending=False).head(n=20)

In [40]:
df_random_pps

,x,y,ppscore,case,is_valid_score,metric,baseline_score,model_score,model
22,charge_off,charge_off,1.000000,predict_itself,True,None,0.0000,1.000000,None
21,active,charge_off,1.000000,regression,True,mean absolute error,0.4866,0.000000,DecisionTreeRegressor()
27,total_writeoff_amt,charge_off,0.999589,regression,True,mean absolute error,0.4866,0.000200,DecisionTreeRegressor()
50,writeoff_type_null,charge_off,0.992613,regression,True,mean absolute error,0.4866,0.003595,DecisionTreeRegressor()
26,principal_amt_chrg_off,charge_off,0.979246,regression,True,mean absolute error,0.4866,0.010099,DecisionTreeRegressor()
28,fee_chg_off_reversal_amt,charge_off,0.871047,regression,True,mean absolute error,0.4866,0.062748,DecisionTreeRegressor()
35,fc_reversals,charge_off,0.840404,regression,True,mean absolute error,0.4866,0.077659,DecisionTreeRegressor()
23,charge_off_aged,charge_off,0.799141,regression,True,mean absolute error,0.4866,0.097738,DecisionTreeRegressor()
36,fee_reversals,charge_off,0.787995,regression,True,mean absolute error,0.4866,0.103161,DecisionTreeRegressor()
18,credit_limit_amt,charge_off,0.681204,regression,True,mean absolute error,0.4866,0.155126,DecisionTreeRegressor()


In [ ]:
final[final['mth'].isnull()]['mth_code'].unique()
#final[final['mth_code'].isnull()]

In [ ]:
final = add_survival_time(final)
final = add_month_nums(final)
final = sort_by_mth_and_snapshot(final)
final = get_uniques(final)
final = final.reset_index()
final.drop('level_0', axis=1, inplace=True)
final.drop('index', axis=1, inplace=True)
final

# Introduction


# Finding ppscore metrics
In traditional data analysis, the first step to determine any underlying relationships is usually finding the correlation matrix of a sample dataframe. However, the correlation matrix has some limitations, mainly the inability to account for non-linear relationships. Predictive power score (PPS) is an alternative to correlation matrix that has the ability to account for non-linear relationships in its scoring. More documentation and explanation can be found here: https://medium.com/geekculture/predictive-power-score-implementation-in-python-70558bf91f45. 

We looked at both correlation and ppscore matrices with relation to their ability to predict whether an account will be charged off. From these results, we saw that ___ columns hasdlfkjsa;ldkfja;sldkfj

In [31]:
final.columns

Index(['financial_active', 'net_payment_behaviour_tripd', 'promotion_flag',
       'variable_rate_index', 'account_status_code', 'active_12_mths',
       'bank_fico_buckets_20', 'mob', 'open_closed_flag',
       'ever_delinquent_flg',
       ...
       'M_FEIAMGTTQ.IUSA', 'M_FXCPMGASUQ.IUSA', 'M_FRBBBQ.IUSA',
       'M_FHOWNRQ.IUSA', 'M_FIFR_FEDS.IUSA', 'M_FIFR$_FEDS.IUSA', 'mth',
       'survival_time', 'month_num', 'max_survival_time'],
      dtype='object', length=168)

In [36]:
data = []
for column in final.columns:
    data.append(pps.score(final, x=column, y="charge_off"))
df_random_pps = pd.DataFrame(data)
df_random_pps = df_random_pps.sort_values(by='ppscore', ascending=False).head(n=20)

In [37]:
df_random_pps

,x,y,ppscore,case,is_valid_score,metric,baseline_score,model_score,model
21,charge_off,charge_off,1.000000,predict_itself,True,None,0.0000,1.000000,None
20,active,charge_off,1.000000,regression,True,mean absolute error,0.0656,0.000000,DecisionTreeRegressor()
26,total_writeoff_amt,charge_off,0.996951,regression,True,mean absolute error,0.0656,0.000200,DecisionTreeRegressor()
49,writeoff_type_null,charge_off,0.894920,regression,True,mean absolute error,0.0656,0.006893,DecisionTreeRegressor()
25,principal_amt_chrg_off,charge_off,0.688517,regression,True,mean absolute error,0.0656,0.020433,DecisionTreeRegressor()
13,prev_balance,charge_off,0.072371,regression,True,mean absolute error,0.0656,0.060852,DecisionTreeRegressor()
109,M_FRGT10Y.IUSA,charge_off,0.000000,regression,True,mean absolute error,0.0656,0.068606,DecisionTreeRegressor()
110,M_FYPDPIQ.IUSA,charge_off,0.000000,regression,True,mean absolute error,0.0656,0.068606,DecisionTreeRegressor()
111,M_FAWHTP.IUSA,charge_off,0.000000,regression,True,mean absolute error,0.0656,0.112757,DecisionTreeRegressor()
112,M_FAHETP.IUSA,charge_off,0.000000,regression,True,mean absolute error,0.0656,0.068606,DecisionTreeRegressor()


# Cox Proportional Hazard Model


The Cox Proportional Hazard Model is a parametric model in survival analysis that takes into account multiple variables to analyze their relationship to their survival distribution.

In [9]:
df_clean_train_final = 
columns = []
dfcox = df_clean_train_final[columns] 
coxph = CoxPHFitter() 
coxph.fit(dfcox, duration_col = 'time', event_col = 'charge_off')
coxph.print_summary()

SyntaxError: invalid syntax (513034295.py, line 1)

In [ ]:
coxph.plot()

In [ ]:
df_ones= dfcox.loc[dfcox['charge_off'] == 1]
df_after = df_ones['time']
coxph.predict_survival_function(df_ones, conditional_after = df_after)

In [ ]:
coxph.check_assumptions(dfcox)

In [ ]:
def findCol(col):
    count = 0
    for column in s.columns:
        if (column == col):
            return count
        count += 1

In [ ]:
predictors = []
rows_to_predict = dfcox.iloc[predictors, []]
rows_to_predict

In [ ]:
survival_function = coxph.predict_survival_function(rows2predict).plot()
survival_function

In [ ]:
start = 
end = 
num_charge_offs = survival_function.loc[start:end, :].diff().iloc[-1].sum() * -1
num_charge_offs